<a href="https://colab.research.google.com/github/Rwolste/DS-3001-Assignments/blob/main/FinanceAPIHW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import requests
import pandas as pd

def fetch_stock_data(symbols):
    url = f"https://yfapi.net/v6/finance/quote?region=US&lang=en&symbols={','.join(symbols)}"
    headers = {"x-api-key": "mNeekuVLCe76wmMeBT7xU1dr0DNCGT5Ljre8w3Ba"}
    response = requests.get(url, headers=headers).json()

    if 'quoteResponse' not in response or 'result' not in response['quoteResponse']:
        return "Error: Invalid response received from API."

    stocks = []
    for stock in response['quoteResponse']['result']:
        stocks.append({
            'Stock Ticker': stock.get('symbol', 'N/A'),
            'Company': stock.get('longName', 'N/A'),
            'Current Market Price': f"${stock.get('regularMarketPrice', 0):.2f}",
            '52 Week High': stock.get('fiftyTwoWeekHigh', 'N/A'),
            '52 Week Low': stock.get('fiftyTwoWeekLow', 'N/A')
        })

    return stocks

def fetch_additional_data(symbol, module):
    url = f"https://yfapi.net/v6/finance/quote?region=US&lang=en&symbols={symbol}"
    headers = {"x-api-key": "mNeekuVLCe76wmMeBT7xU1dr0DNCGT5Ljre8w3Ba"}
    response = requests.get(url, headers=headers).json()

    if 'quoteResponse' not in response or 'result' not in response['quoteResponse']:
        return "Error: Invalid response received from API."

    stock_data = response['quoteResponse']['result'][0]
    data = {
        '52 Week High': stock_data.get('fiftyTwoWeekHigh', 'N/A'),
        '52 Week Low': stock_data.get('fiftyTwoWeekLow', 'N/A'),
        'Return on Assets (ROA)': stock_data.get('returnOnAssets', 'N/A')
    }
    if module == '52 Week High':
        data = {'52 Week High': stock_data.get('fiftyTwoWeekHigh', 'N/A')}

    elif module == '52 Week Low':
        data = {'52 Week Low': stock_data.get('fiftyTwoWeekLow', 'N/A')}

    elif module == 'Return on Assets':
        data = {'Return on Assets (ROA)': stock_data.get('returnOnAssets', 'N/A')}

    df = pd.DataFrame([data])
    return df

def fetch_trending_stocks():
    url = "https://yfapi.net/v1/finance/trending/US"
    headers = {"x-api-key": "mNeekuVLCe76wmMeBT7xU1dr0DNCGT5Ljre8w3Ba"}
    response = requests.get(url, headers=headers).json()

    if 'finance' not in response or 'result' not in response['finance']:
        return "Error: Invalid response received from API."

    trending_symbols = [stock['symbol'] for stock in response['finance']['result'][0].get('quotes', [])[:5]]

    return fetch_stock_data(trending_symbols)

def main():
    symbols = input("Enter stock symbols separated by commas: ").split(',')
    stock_data = fetch_stock_data(symbols)
    if isinstance(stock_data, str):
      print(stock_data)

    else:
      for stock in stock_data:
            print(f"Stock Ticker: {stock['Stock Ticker']}, Company: {stock['Company']}, Current Market Price: {stock['Current Market Price']}, 52 Week High: {stock['52 Week High']}, 52 Week Low: {stock['52 Week Low']}")

    symbol = input("Enter a stock symbol for additional data: ")
    module = input("Choose module (52 Week High, 52 Week Low, Return on Assets): ")
    df = fetch_additional_data(symbol, module)
    print(df)

    print("Trending Stocks:")
    trending_data = fetch_trending_stocks()
    if isinstance(trending_data, str):
        print(trending_data)
    else:
        for stock in trending_data:
            print(f"Stock Ticker: {stock['Stock Ticker']}, Company: {stock['Company']}, Current Market Price: {stock['Current Market Price']}, 52 Week High: {stock['52 Week High']}, 52 Week Low: {stock['52 Week Low']}")

if __name__ == "__main__":
    main()

Enter stock symbols separated by commas: tsla
Stock Ticker: TSLA, Company: Tesla, Inc., Current Market Price: $290.80, 52 Week High: 488.54, 52 Week Low: 138.8
Enter a stock symbol for additional data: tsla
Choose module (52 Week High, 52 Week Low, Return on Assets): r
   52 Week High  52 Week Low Return on Assets (ROA)
0        488.54        138.8                    N/A
Trending Stocks:
Stock Ticker: NVDA, Company: NVIDIA Corporation, Current Market Price: $131.28, 52 Week High: 153.13, 52 Week Low: 75.606
Stock Ticker: CRM, Company: Salesforce, Inc., Current Market Price: $307.33, 52 Week High: 369.0, 52 Week Low: 212.0
Stock Ticker: SNOW, Company: Snowflake Inc., Current Market Price: $166.19, 52 Week High: 194.4, 52 Week Low: 107.13
Stock Ticker: IONQ, Company: IonQ, Inc., Current Market Price: $29.93, 52 Week High: 54.74, 52 Week Low: 6.22
Stock Ticker: BYND, Company: Beyond Meat, Inc., Current Market Price: $3.56, 52 Week High: 11.7, 52 Week Low: 3.3
